In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import glob

%matplotlib inline

pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
#path = r'C:\Users\eangenello\Desktop\ECM'
#allFiles = glob.glob(path + "/*.xls")
#data = pd.DataFrame()
#list_ = []
#for file_ in allFiles:
#    df = pd.read_excel(file_,index_col=None, header=0)
#    list_.append(df)
#data = pd.concat(list_)

In [3]:
data = pd.read_excel(r'C:\Users\Erika\Desktop\PYTHON\output\Balance\ECM histórico.xlsx')

In [4]:
data.head()

,cod_cia,Segmento,Periodo,DenominacionCorta,Estado,CM_requerido,Ccomputable,Def_Sup,juicios
0,822,Personas_s/RM,2012-2,INSTITUTO DE JUJUY,Activa,"4,000,000.00",4967976,"967,976.00",0
1,823,Patrimoniales_Mixtas,2012-2,IÚNIGO,Activa,"10,000,000.00",10075752,"75,752.00",0
2,2,Automotores,2012-3,SMG,Activa,"51,611,742.60",137548535,"85,936,792.40",1893
3,6,Personas_s/RM,2012-3,SURCO,Activa,"13,688,651.76",18561948,"4,873,296.24",411
4,7,Patrimoniales_Mixtas,2012-3,CAMINOS PROTEGIDOS,Activa,"18,000,000.00",25936426,"7,936,426.00",16


In [5]:
'''data = data.fillna(0)
data = data.replace(to_replace='vacio', value=0)'''

"data = data.fillna(0)\ndata = data.replace(to_replace='vacio', value=0)"

In [6]:
#data['Periodo'] = data['Año'].astype(str).str.cat(data['Trimestre'].astype(str), sep='-')

In [7]:
data['Deficit']= np.where(data['Def_Sup'] < 0, 1, 0)

In [8]:
data

,cod_cia,Segmento,Periodo,DenominacionCorta,Estado,CM_requerido,Ccomputable,Def_Sup,juicios,Deficit
0,822,Personas_s/RM,2012-2,INSTITUTO DE JUJUY,Activa,"4,000,000.00",4967976,"967,976.00",0,0
1,823,Patrimoniales_Mixtas,2012-2,IÚNIGO,Activa,"10,000,000.00",10075752,"75,752.00",0,0
2,2,Automotores,2012-3,SMG,Activa,"51,611,742.60",137548535,"85,936,792.40",1893,0
3,6,Personas_s/RM,2012-3,SURCO,Activa,"13,688,651.76",18561948,"4,873,296.24",411,0
4,7,Patrimoniales_Mixtas,2012-3,CAMINOS PROTEGIDOS,Activa,"18,000,000.00",25936426,"7,936,426.00",16,0
...,...,...,...,...,...,...,...,...,...,...
5230,888,Personas_s/RM,2019-2,PEUGEOT CITROËN,Activa,"56,057,414.00",182388583,"126,331,169.00",0,0
5231,890,ART,2019-2,MUTUAL PETROLEROS ART,Activa,"60,000,000.00",96145750,"36,145,750.00",0,0
5232,894,Personas_s/RM,2019-2,OMINT,Activa,"13,500,000.00",18970406,"5,470,406.00",0,0
5233,895,Créditos/Caución,2019-2,PACÍFICO,Activa,"13,500,000.00",21757222,"8,257,222.00",1,0


In [9]:
data['periodo2'] = data['Periodo'].apply(lambda x: int(x.replace('-','0')))

In [10]:
data.sort_values(by = ['cod_cia','periodo2'], inplace = True)

In [11]:
prueba=data.copy()

In [12]:
#Trabajar con los incrementos
ind_a_incr = ['Def_Sup']
cols = prueba.columns

grupo_inc = prueba.groupby('cod_cia')

for p in [1,2,3,4]:
    sufijo = str(p)
    for i in ind_a_incr:
        nombre = i+'_'+sufijo
        prueba[nombre] = grupo_inc[i].pct_change(periods = int(sufijo))
        del nombre
    del sufijo

#Ver deficit
prueba_inv = prueba[cols].sort_values(by = ['cod_cia','Periodo'], ascending = [True,False]).copy()
grupo_inv = prueba_inv.groupby('cod_cia')['Def_Sup']

for p in [1,2,3,4]:
    nombre = 'solv_'+str(p)
    prueba_inv[nombre] = grupo_inv.shift(int(p))
    del nombre
    
#Reordenar nuevamente las filas
prueba_inv.sort_values(by = ['cod_cia','Periodo'], inplace = True)
prueba_inv.reset_index(drop = True, inplace = True)

#Generar la columna de déficit o superavit de CM para el año siguiente de observacion. 1 Deficit 0 Superavit

prueba_inv['sup_def'] = np.where((prueba_inv['solv_1']<0)|(prueba_inv['solv_2']<0)|(prueba_inv['solv_3']<0)|
                                 (prueba_inv['solv_4']<0),1,0)


In [13]:
prueba_inv

,cod_cia,Segmento,Periodo,DenominacionCorta,Estado,CM_requerido,Ccomputable,Def_Sup,juicios,Deficit,periodo2,solv_1,solv_2,solv_3,solv_4,sup_def
0,2,Automotores,2012-2,SMG,Activa,"44,977,809.73",157205955,"112,228,145.27",1897,0,201202,"85,936,792.40","86,776,403.84","112,824,645.53","86,787,440.21",0
1,2,Automotores,2012-3,SMG,Activa,"51,611,742.60",137548535,"85,936,792.40",1893,0,201203,"86,776,403.84","112,824,645.53","86,787,440.21","105,302,504.39",0
2,2,Automotores,2012-4,SMG,Activa,"56,293,185.16",143069589,"86,776,403.84",1748,0,201204,"112,824,645.53","86,787,440.21","105,302,504.39","109,814,448.14",0
3,2,Automotores,2013-1,SMG,Activa,"60,890,032.47",173714678,"112,824,645.53",1736,0,201301,"86,787,440.21","105,302,504.39","109,814,448.14","148,700,880.12",0
4,2,Automotores,2013-2,SMG,Activa,"67,300,492.79",154087933,"86,787,440.21",1827,0,201302,"105,302,504.39","109,814,448.14","148,700,880.12","213,538,791.71",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230,895,Créditos/Caución,2018-3,PACÍFICO,Activa,"15,750,000.00",24665081,"8,915,081.00",0,0,201803,"15,385,105.00","14,864,868.00","8,257,222.00",nan,0
5231,895,Créditos/Caución,2018-4,PACÍFICO,Activa,"9,000,000.00",24385105,"15,385,105.00",0,0,201804,"14,864,868.00","8,257,222.00",nan,nan,0
5232,895,Créditos/Caución,2019-1,PACÍFICO,Activa,"11,250,000.00",26114868,"14,864,868.00",0,0,201901,"8,257,222.00",nan,nan,nan,0
5233,895,Créditos/Caución,2019-2,PACÍFICO,Activa,"13,500,000.00",21757222,"8,257,222.00",1,0,201902,nan,nan,nan,nan,0


In [14]:
a = prueba_inv.copy()
a['Deficit']= a['Deficit'].astype(str)

prueba_inv.pivot_table('Periodo', ['Deficit', 'periodo2'], aggfunc='count')

a.to_excel(r'C:\Users\Erika\Desktop\PYTHON\output\Distribucion_deficit.xlsx')

In [15]:
prueba_inv.columns

Index(['cod_cia', 'Segmento', 'Periodo', 'DenominacionCorta', 'Estado',
       'CM_requerido', 'Ccomputable', 'Def_Sup', 'juicios', 'Deficit',
       'periodo2', 'solv_1', 'solv_2', 'solv_3', 'solv_4', 'sup_def'],
      dtype='object')

In [16]:
#Juntar el set original con la columna de sup_deficit
#conjunto = prueba.copy()
#conjunto['sup_def'] = prueba_inv['sup_def']

#Generamos la base final únicamente cuando el período de observación no tiene déficit y en el año siguiente SI
#base = conjunto.loc[~((conjunto['Def_Sup']<0) & (conjunto['sup_def']==1)) & 
#           (conjunto['periodo2']<201803) & (conjunto['periodo2']>201202)].copy()

#base.reset_index(drop = True, inplace = True)
    
#maximo_reemplazo = {i:base[base[i]!= np.inf][i].max() for i in base.columns[10:]}
#minimo_reemplazo = {i:base[base[i]!= -np.inf][i].min() for i in base.columns[10:]}

#for i in base.columns[10:]:
#    base[i] = base[i].replace(np.inf,maximo_reemplazo[i])
#    base[i] = base[i].replace(-np.inf,minimo_reemplazo[i])

In [17]:
base=prueba_inv.copy()
base = base.rename(columns={'Periodo':'periodo'})

In [18]:
base.head()

,cod_cia,Segmento,periodo,DenominacionCorta,Estado,CM_requerido,Ccomputable,Def_Sup,juicios,Deficit,periodo2,solv_1,solv_2,solv_3,solv_4,sup_def
0,2,Automotores,2012-2,SMG,Activa,"44,977,809.73",157205955,"112,228,145.27",1897,0,201202,"85,936,792.40","86,776,403.84","112,824,645.53","86,787,440.21",0
1,2,Automotores,2012-3,SMG,Activa,"51,611,742.60",137548535,"85,936,792.40",1893,0,201203,"86,776,403.84","112,824,645.53","86,787,440.21","105,302,504.39",0
2,2,Automotores,2012-4,SMG,Activa,"56,293,185.16",143069589,"86,776,403.84",1748,0,201204,"112,824,645.53","86,787,440.21","105,302,504.39","109,814,448.14",0
3,2,Automotores,2013-1,SMG,Activa,"60,890,032.47",173714678,"112,824,645.53",1736,0,201301,"86,787,440.21","105,302,504.39","109,814,448.14","148,700,880.12",0
4,2,Automotores,2013-2,SMG,Activa,"67,300,492.79",154087933,"86,787,440.21",1827,0,201302,"105,302,504.39","109,814,448.14","148,700,880.12","213,538,791.71",0


In [19]:
balance=pd.read_excel(r'C:\Users\Erika\Desktop\PYTHON\output\Balance\Dt_Balance_final.xlsx')

In [20]:
balance.head()

,cod_cia,periodo,1.00.00.00.00.00.00.00,1.01.00.00.00.00.00.00,1.01.01.00.00.00.00.00,1.01.01.01.00.00.00.00,1.01.01.01.01.00.00.00,1.01.01.01.01.01.00.00,1.01.01.01.01.02.00.00,1.01.01.01.01.03.00.00,...,5.02.04.04.00.00.00.00,5.02.04.04.01.00.00.00,5.02.04.04.01.01.00.00,5.02.04.04.02.00.00.00,5.02.04.04.02.01.00.00,5.02.04.04.03.00.00.00,5.02.04.04.03.01.00.00,5.02.04.04.04.00.00.00,5.02.04.04.04.50.00.00,5.03.00.00.00.00.00.00
0,2,2012-2,566593246,11885274,3250465,3250465,3250465,278167,60500,2911798,...,32057,0,0,0,0,0,0,32057,32057,0
1,2,2012-3,586676265,11918947,3233892,3233892,3233892,177962,68500,2987430,...,153432,0,0,0,0,0,0,153432,153432,0
2,2,2012-4,625268985,11902870,2818391,2818391,2818391,130288,71500,2616603,...,84449,0,0,0,0,0,0,84449,84449,0
3,2,2013-1,674591662,7110632,6717952,6717952,6717952,171122,84000,6462830,...,32464,0,0,0,0,0,0,32464,32464,0
4,2,2013-2,740195439,19484948,2838685,2838685,2838685,134419,84000,2620266,...,34666,0,0,0,0,0,0,34666,34666,0


In [21]:
base_final = base.merge(balance, how='left', on=['cod_cia', 'periodo'])

In [22]:
base_final.head()

,cod_cia,Segmento,periodo,DenominacionCorta,Estado,CM_requerido,Ccomputable,Def_Sup,juicios,Deficit,...,5.02.04.04.00.00.00.00,5.02.04.04.01.00.00.00,5.02.04.04.01.01.00.00,5.02.04.04.02.00.00.00,5.02.04.04.02.01.00.00,5.02.04.04.03.00.00.00,5.02.04.04.03.01.00.00,5.02.04.04.04.00.00.00,5.02.04.04.04.50.00.00,5.03.00.00.00.00.00.00
0,2,Automotores,2012-2,SMG,Activa,"44,977,809.73",157205955,"112,228,145.27",1897,0,...,"32,057.00",0.00,0.00,0.00,0.00,0.00,0.00,"32,057.00","32,057.00",0.00
1,2,Automotores,2012-3,SMG,Activa,"51,611,742.60",137548535,"85,936,792.40",1893,0,...,"153,432.00",0.00,0.00,0.00,0.00,0.00,0.00,"153,432.00","153,432.00",0.00
2,2,Automotores,2012-4,SMG,Activa,"56,293,185.16",143069589,"86,776,403.84",1748,0,...,"84,449.00",0.00,0.00,0.00,0.00,0.00,0.00,"84,449.00","84,449.00",0.00
3,2,Automotores,2013-1,SMG,Activa,"60,890,032.47",173714678,"112,824,645.53",1736,0,...,"32,464.00",0.00,0.00,0.00,0.00,0.00,0.00,"32,464.00","32,464.00",0.00
4,2,Automotores,2013-2,SMG,Activa,"67,300,492.79",154087933,"86,787,440.21",1827,0,...,"34,666.00",0.00,0.00,0.00,0.00,0.00,0.00,"34,666.00","34,666.00",0.00


In [23]:
#base_final.to_csv(r'C:\Users\Erika\Desktop\PYTHON\output\Balance\Dataset_final.csv', index=False)

In [24]:
column_names = base_final.columns.values
column_names

array(['cod_cia', 'Segmento', 'periodo', 'DenominacionCorta', 'Estado',
       'CM_requerido', 'Ccomputable', 'Def_Sup', 'juicios', 'Deficit',
       'periodo2', 'solv_1', 'solv_2', 'solv_3', 'solv_4', 'sup_def',
       '1.00.00.00.00.00.00.00', '1.01.00.00.00.00.00.00',
       '1.01.01.00.00.00.00.00', '1.01.01.01.00.00.00.00',
       '1.01.01.01.01.00.00.00', '1.01.01.01.01.01.00.00',
       '1.01.01.01.01.02.00.00', '1.01.01.01.01.03.00.00',
       '1.01.02.00.00.00.00.00', '1.01.02.01.00.00.00.00',
       '1.01.02.01.01.00.00.00', '1.01.02.01.01.01.00.00',
       '1.01.02.01.01.02.00.00', '1.01.02.01.01.50.00.00',
       '1.01.02.01.02.00.00.00', '1.01.02.01.02.01.00.00',
       '1.01.02.01.02.02.00.00', '1.01.02.01.02.50.00.00',
       '1.01.03.00.00.00.00.00', '1.01.03.01.00.00.00.00',
       '1.01.03.01.01.00.00.00', '1.01.03.01.02.00.00.00',
       '1.02.00.00.00.00.00.00', '1.02.01.00.00.00.00.00',
       '1.02.01.01.00.00.00.00', '1.02.01.01.01.00.00.00',
       '1.02.01.01.0

In [25]:
seg_dummys = pd.get_dummies(base_final['Segmento'], prefix = 'Segmento')

In [26]:
seg_dummys

,Segmento_ART,Segmento_Automotores,Segmento_Créditos/Caución,Segmento_Patrimoniales_Mixtas,Segmento_Personas_s/RM,Segmento_Retiro,Segmento_TPP,Segmento_Vida_RM
0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
5230,0,0,1,0,0,0,0,0
5231,0,0,1,0,0,0,0,0
5232,0,0,1,0,0,0,0,0
5233,0,0,1,0,0,0,0,0


In [27]:
df_new = base_final[column_names].join(seg_dummys)

In [28]:
df_new

,cod_cia,Segmento,periodo,DenominacionCorta,Estado,CM_requerido,Ccomputable,Def_Sup,juicios,Deficit,...,5.02.04.04.04.50.00.00,5.03.00.00.00.00.00.00,Segmento_ART,Segmento_Automotores,Segmento_Créditos/Caución,Segmento_Patrimoniales_Mixtas,Segmento_Personas_s/RM,Segmento_Retiro,Segmento_TPP,Segmento_Vida_RM
0,2,Automotores,2012-2,SMG,Activa,"44,977,809.73",157205955,"112,228,145.27",1897,0,...,"32,057.00",0.00,0,1,0,0,0,0,0,0
1,2,Automotores,2012-3,SMG,Activa,"51,611,742.60",137548535,"85,936,792.40",1893,0,...,"153,432.00",0.00,0,1,0,0,0,0,0,0
2,2,Automotores,2012-4,SMG,Activa,"56,293,185.16",143069589,"86,776,403.84",1748,0,...,"84,449.00",0.00,0,1,0,0,0,0,0,0
3,2,Automotores,2013-1,SMG,Activa,"60,890,032.47",173714678,"112,824,645.53",1736,0,...,"32,464.00",0.00,0,1,0,0,0,0,0,0
4,2,Automotores,2013-2,SMG,Activa,"67,300,492.79",154087933,"86,787,440.21",1827,0,...,"34,666.00",0.00,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230,895,Créditos/Caución,2018-3,PACÍFICO,Activa,"15,750,000.00",24665081,"8,915,081.00",0,0,...,146.00,0.00,0,0,1,0,0,0,0,0
5231,895,Créditos/Caución,2018-4,PACÍFICO,Activa,"9,000,000.00",24385105,"15,385,105.00",0,0,...,"390,434.00",0.00,0,0,1,0,0,0,0,0
5232,895,Créditos/Caución,2019-1,PACÍFICO,Activa,"11,250,000.00",26114868,"14,864,868.00",0,0,...,"352,228.00",0.00,0,0,1,0,0,0,0,0
5233,895,Créditos/Caución,2019-2,PACÍFICO,Activa,"13,500,000.00",21757222,"8,257,222.00",1,0,...,"-180,047.00",0.00,0,0,1,0,0,0,0,0


In [29]:
df_new = df_new.rename(columns={'sup_def':'target'})

In [30]:
df_new = df_new.drop({'Estado','Def_Sup','periodo2'},1)
#,'Def_Sup_1','Def_Sup_2','Def_Sup_3','Def_Sup_4'

In [31]:
df_new.head()

,cod_cia,Segmento,periodo,DenominacionCorta,CM_requerido,Ccomputable,juicios,Deficit,solv_1,solv_2,...,5.02.04.04.04.50.00.00,5.03.00.00.00.00.00.00,Segmento_ART,Segmento_Automotores,Segmento_Créditos/Caución,Segmento_Patrimoniales_Mixtas,Segmento_Personas_s/RM,Segmento_Retiro,Segmento_TPP,Segmento_Vida_RM
0,2,Automotores,2012-2,SMG,"44,977,809.73",157205955,1897,0,"85,936,792.40","86,776,403.84",...,"32,057.00",0.00,0,1,0,0,0,0,0,0
1,2,Automotores,2012-3,SMG,"51,611,742.60",137548535,1893,0,"86,776,403.84","112,824,645.53",...,"153,432.00",0.00,0,1,0,0,0,0,0,0
2,2,Automotores,2012-4,SMG,"56,293,185.16",143069589,1748,0,"112,824,645.53","86,787,440.21",...,"84,449.00",0.00,0,1,0,0,0,0,0,0
3,2,Automotores,2013-1,SMG,"60,890,032.47",173714678,1736,0,"86,787,440.21","105,302,504.39",...,"32,464.00",0.00,0,1,0,0,0,0,0,0
4,2,Automotores,2013-2,SMG,"67,300,492.79",154087933,1827,0,"105,302,504.39","109,814,448.14",...,"34,666.00",0.00,0,1,0,0,0,0,0,0


In [32]:
df_new.to_excel(r'C:\Users\Erika\Desktop\PYTHON\output\Balance\df_new.xlsx', index=False)